This notebook measures the intensities and errors of CO 2-1 emission at the locations of all of the SNe in our sample across all of the resolutions in the survey. This is used to prepare to answer the question in section 3.1 Do we see gas at supernova locations? 

We will use this to uncover what resolution will be best for this study.

In [1]:
# IMPORT PACKAGES

import numpy as np
from matplotlib import pyplot as plt
import astropy.io.fits as pyfits
from astropy.table import Table, join
from astropy.wcs import WCS
from astropy.io import ascii
import astropy
from astropy.io import fits

In [13]:
# import SNeCO data

dataFile = '../Data/3.SNe+GalData.csv'
dataTable = Table.read(dataFile, format='csv') 
dataTable.colnames

dataTable

galaxy,SN_name,SN_type,SN_ra,SN_dec,SN_xCoord,SN_yCoord,SN_co21int,SN_co21err,SN_co21ew,map_beamsize,orient_ra,orient_dec,dist,orient_posang,orient_incl,size_r25
str8,str11,str12,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
circinus,SN1996cr,II,213.29176388888888,-65.34570462962962,86.56761411031889,84.63433198287285,186.84804,2.137819,34.56537,0.002032163908078,213.29124,-65.33915,4.2,36.74,64.3,262.49515
ngc0253,SN1940E,I,11.878291666666666,-25.293444444444447,296.9010534051223,236.26662905273994,218.98796,1.4441332,45.862267,0.00232543637022,11.887966,-25.288443,3.7,52.48,75.0,803.7504
ngc0300,SN2019qyl,II,13.739875,-37.644441666666665,124.97990204934794,202.45166439134732,-0.5273654,0.45072383,nan,0.002271061982879,13.723024,-37.684475,2.09,114.3,39.8,582.26587
ngc0628,SN2019krl,II,24.2067875,15.77946388888889,227.48504797816918,732.1448775793053,1.3992676,1.0197889,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592
ngc0628,SN2013ej,II,24.200666666666667,15.758611111111112,333.46022572413716,356.77980642079694,0.9655536,1.4078557,nan,0.0003114595189268,24.173855,15.783643,9.84,20.7,8.9,296.56592
ngc1087,SN1995V,II,41.61152777777777,-0.4987861111111111,161.4198245203906,287.6499560537321,11.794075,1.0422765,5.175495,0.0004451676280474,41.60492,-0.498717,15.85,359.1,42.9,89.14999
ngc1097,SN1992bd,II,41.579177083333335,-30.275555555555556,291.5099030167368,315.426803230518,296.06226,2.0188875,45.82589,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452
ngc1097,SN1999eu,II,41.58662291666667,-30.318361111111113,245.2470996170838,7.223039949207134,0.0,0.7541714,nan,0.0004711325009833,41.578957,-30.274675,13.58,122.4,48.6,317.0452
ngc1365,SN1983V,Ic,53.381876388888884,-36.14859166666667,571.5153905023813,127.96380165601636,5.781725,1.1458964,3.55028,0.0003829740084505,53.40152,-36.140404,19.57,201.1,55.4,360.67926


In [12]:
# pull galaxy names to generate the file lists

galaxies = dataTable['galaxy']

 galaxy 
--------
circinus
 ngc0253
 ngc0300
 ngc0628
 ngc0628
 ngc1087
 ngc1097
 ngc1097
 ngc1365
 ngc1365
     ...
 ngc4424
 ngc4457
ngc4496a
ngc4496a
 ngc4536
 ngc4579
 ngc4579
 ngc5128
 ngc5128
 ngc5530
 ngc6744
Length = 58 rows


In [5]:
# Next take measurements for Native Resolution
# NatFile (= Prefix), NatBeamSize, NatRes, NatInt, NatErr, NatEW

#Compile file lists
prefixArr, NatIntFile, NatErrFile, NatEWFile = [],[],[],[]
pc60IntFile,   pc60ErrFile,   pc60EWFile   = [],[],[]
pc90IntFile,   pc90ErrFile,   pc90EWFile   = [],[],[]
pc120IntFile,  pc120ErrFile,  pc120EWFile  = [],[],[]
pc150IntFile,  pc150ErrFile,  pc150EWFile  = [],[],[]
pc500IntFile,  pc500ErrFile,  pc500EWFile  = [],[],[]
pc750IntFile,  pc750ErrFile,  pc750EWFile  = [],[],[]
pc1000IntFile, pc1000ErrFile, pc1000EWFile = [],[],[]
pc1250IntFile, pc1250ErrFile, pc1250EWFile = [],[],[]

IntFileNames = [NatIntFile, pc60IntFile, pc90IntFile, pc120IntFile, pc150IntFile, pc500IntFile, pc750IntFile, pc1000IntFile, pc1250IntFile]
ErrFileNames = [NatErrFile, pc60ErrFile, pc90ErrFile, pc120ErrFile, pc150ErrFile, pc500ErrFile, pc750ErrFile, pc1000ErrFile, pc1250ErrFile]
EWFileNames  = [NatEWFile, pc60EWFile, pc90EWFile, pc120EWFile, pc150EWFile, pc500EWFile, pc750EWFile, pc1000EWFile, pc1250EWFile]

# /data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/ngc2997/ngc2997_12m+7m+tp_co21_broad_mom0.fits

intType = "_co21_broad_mom0"
errType = "_co21_broad_emom0"
ewType  = "_co21_strict_ew"
preamble = "/data/tycho/0/leroy.42/reduction/alma/phangs-alma/derived/"


for i in range(len(Gal)):
    
    GalName = Gal[i].strip()
    
    if GalName == 'circinus' or GalName == 'ngc0247' or GalName == 'ngc0253' or GalName == 'ngc0300' or GalName == 'ngc5128':
        prefix = "7m+tp"
#     elif GalName == 'ngc2775':
#         prefix = "12m+7m
    else:
        prefix = "12m+7m+tp"
        
    prefixArr.append(prefix)
    
    res = ['', '_60pc', '_90pc', '_120pc', '_150pc', '_500pc', '_750pc', '_1000pc', '_1250pc']
    
    for i in range(len(res)):
        
        IntPathString = preamble + GalName + "/" + GalName + "_" + prefix + intType + res[i] + ".fits"
        ErrPathString = preamble + GalName + "/" + GalName + "_" + prefix + errType + res[i] + ".fits"
        EWPathString  = preamble + GalName + "/" + GalName + "_" + prefix + ewType + res[i] + ".fits"

        if(os.path.isfile(IntPathString) == True):
            IntFileNames[i].append(IntPathString)
        
        else:
            print("No Int file for ", + GalName[i])
        
        if(os.path.isfile(ErrPathString) == True):
            ErrFileNames[i].append(ErrPathString)
        
        else:
            print("No Err file for ", + GalName[i])
            
        if(os.path.isfile(EWPathString) == True):
            EWFileNames[i].append(EWPathString) 
        
        else:
            print("No EW file for ", + GalName[i])
        
print(IntFileNames[0][0])


# #Set up alphaCOlist

# alphaCOlist = []

# for i in range(len(Gal)):
    
#     GalName = Gal[i].strip().upper()
#     if GalName == 'CIRCINUS':
#         GalName = 'ESO097-013'
    
#     alphaCOstring = "../alphaCOmaps/" + GalName + '_alphaCO21_Sun+20.fits'
#     alphaCOlist.append(alphaCOstring)
# # print(alphaCOlist)


['galaxy',
 'SN_name',
 'SN_type',
 'SN_ra',
 'SN_dec',
 'SN_xCoord',
 'SN_yCoord',
 'SN_co21int',
 'SN_co21err',
 'SN_co21ew',
 'map_beamsize',
 'orient_ra',
 'orient_dec',
 'dist',
 'orient_posang',
 'orient_incl',
 'size_r25']